# Capstone Project - The Battle of Neighborhoods (Week 4)

### Table of Contents:
#### 1.Business Problem,Background Description.
#### 2.Data Description,Using data to solve the problem.

####  Background Description.
An IT Professional wants to start Indian Grocery Store Business in NC. 
He is the resident of NC since many years and wants to know about the best locations to start the Business where he can earn more profits as expected.

#### Business Problem.
The business problem is to determine the best location based on geographic information to start an Indian Grocery Store business, where more profits are earned. 



#### Data Description.
The Client wants to start Business in anyone of the four cities like Morrisville, Cary, Durham, Raleigh.
The Zipcodes including postal codes of these cities are as follows: Morrisville = 27560 , Cary = 27511 , Durham = 27707, Raleigh = 27606

#### Loading Libraries

In [1]:
# Loading libraries

import numpy as np
import pandas as pd

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

import json

from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

from bs4 import BeautifulSoup

import geocoder 
import copy
import shapely.geometry
import pyproj
import math

print('Importing libraries completed.')

Importing libraries completed.


In [12]:
col1 = ['Location Name','Latitude','Longitude']

loc = pd.DataFrame(columns = col1)
loc['Location Name'] = ['Parkwoods','Victoria Village','Harbourfront','Lawrence Heights']
loc['Postal Code'] = ['00601','00602','00603','00606']
loc

,Location Name,Latitude,Longitude,Postal Code
0,Parkwoods,NaN,NaN,00601
1,Victoria Village,NaN,NaN,00602
2,Harbourfront,NaN,NaN,00603
3,Lawrence Heights,NaN,NaN,00606


In [13]:
google_api_key = 'masked'

def get_coordinates(api_key, address, verbrose=False):
    try:
        url = "https://gist.github.com/erichurst/7882666".format(address, api_key)
        response=requests.get(url).json()
        if verbose:
            print('https://gist.github.com/erichurst/7882666', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location']
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return[lat,lon]
    except:
        return [None, None]
a=0
for i in loc['Location Name']:
        lat, lon = get_coordinates(google_api_key, i)
        loc.iloc[a,2] = lat
        loc.iloc[a,3] = lon
        a= a +1
        
loc

col= ['Location Name','Zipcode','Latitude','Longitude']

loc= pd.DataFrame(columns = col)
loc['Location Name'] = ['Parkwoods','Victoria Village','Harbourfront','Lawrence Heights']
loc['Zipcode'] = ['00601','00602','00603','00606']
loc['Latitude'] = ['18.180555','18.361945','18.455183','18.158345']
loc['Longitude'] = ['-66.749961','-67.175597','-67.119887','-66.932911']
loc


,Location Name,Zipcode,Latitude,Longitude
0,Parkwoods,00601,18.180555,-66.749961
1,Victoria Village,00602,18.361945,-67.175597
2,Harbourfront,00603,18.455183,-67.119887
3,Lawrence Heights,00606,18.158345,-66.932911


In [14]:
CLIENT_ID = 'MS' 
CLIENT_SECRET = 'MS'
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MS
CLIENT_SECRET:MS


In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=6000, LIMIT=400):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lng,
        radius,LIMIT)
        
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        venues_list.append([(
        
              name,
              lat,
              lng,
              v['venue']['name'],
              v['venue']['location']['lat'],
              v['venue']['location']['lng'],
              v['venue']['categories'][0]['name']) for v in results])
        
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['postcode',
                                 'NeighborhoodLatitude',
                                 'NeighborhoodLongitude',
                                 'Venue',
                                 'VenueLatitude',
                                 'VenueLongitude',
                                 'VenueCategory']
        return(nearby_venues)
                            
                                                                                                                                        

In [27]:
#NC_venues = getNearbyVenues(names=loc['Location Name'],
 #                          latitudes=loc['Latitude'],
  #                         longitudes=loc['Longitude']
   #                        )
loc= pd.DataFrame(columns = col)
loc['Location Name'] = ['Parkwoods','Victoria Village','Harbourfront','Lawrence Heights']
loc['state'] = ['NC','NC','NC','NC']
loc

,Location Name,Zipcode,Latitude,Longitude,state
0,Parkwoods,NaN,NaN,NaN,NC
1,Victoria Village,NaN,NaN,NaN,NC
2,Harbourfront,NaN,NaN,NaN,NC
3,Lawrence Heights,NaN,NaN,NaN,NC


In [30]:
len(loc)

4

#### Clustering

In [31]:
def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj='utm',zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj='utm',zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

    

In [21]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import bs4
import lxml.html as lh

In [94]:
url = 'https://gist.github.com/erichurst/7882666'
req = requests.get(url)

In [95]:
soup = bs4.BeautifulSoup(req.text,'html5lib')

In [96]:
data = soup.select('.wikitable.sortable')
print(type(data))
print(len(data))

<class 'list'>
0


In [97]:
doc = lh.fromstring(req.content)
tr_elements = doc.xpath('//tr')
[len(T) for T in tr_elements[:12]]

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [98]:
tr_elements = doc.xpath('//tr')
col=[]
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))

In [99]:
for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    if len(T)!=3:
        break
    i=0
    
    for t in T.iterchildren():
        data=t.text_content()
        col[i][1].append(data)
        i+=1

In [101]:
# Dataframe creation form List

Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)



In [ ]:
for i in range(0,211):
    df.iloc[i,2] = df.iloc[i,2].strip('\n')

In [78]:
df = df.reset_index(drop = True)

In [79]:
for i in range(0,211):
    if df.iloc[i,2] == 'Not assigned':
        df.iloc[i,2] = df.iloc[i,1]

In [45]:
df.head(5)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [80]:
df1 = (df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(set(x.dropna()))).reset_index())
df1.shape

(103, 3)

In [46]:
loc= pd.DataFrame(columns = col)
loc['Location Name'] = ['Parkwoods','Victoria Village','Harbourfront','Lawrence Heights']
loc['Zipcode'] = ['00601','00602','00603','00606']
loc['Latitude'] = ['18.180555','18.361945','18.455183','18.158345']
loc['Longitude'] = ['-66.749961','-67.175597','-67.119887','-66.932911']
loc


,Location Name,Zipcode,Latitude,Longitude
0,Parkwoods,00601,18.180555,-66.749961
1,Victoria Village,00602,18.361945,-67.175597
2,Harbourfront,00603,18.455183,-67.119887
3,Lawrence Heights,00606,18.158345,-66.932911


Data is Cleaned and about to have Caculations